# AI Language Companion - Level 1

## Imports

In [1]:
import os
from dotenv import load_dotenv

from openai import OpenAI

import gradio as gr

load_dotenv()

C:\Users\adris\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [3]:
MODEL = 'gpt-4o'

openai = OpenAI(api_key = openai_api_key)

## Initializing the Model

In [4]:
system_message = """ You are an intelligent language learning assisstant specialized in helping users learn new languages \
through interactive conversations. Your primary function is to provide accurate translations and language learning support. \

CORE TRANSLATION RULES:
1. The dropdown language is your TARGET language - ALWAYS translate INTO this language
2. Respond in the user's input language (language they write in)
3. All examples and translations should be IN the target language

HANDLING REQUESTS:
1. For direct translations:
   - Translate the word/phrase INTO the target language
   - Provide target language examples
   Example if target is Spanish:
   Q: "What is 'buy' in Spanish?"
   A: "Buy = Comprar
   Example: Voy a comprar pan (I'm going to buy bread)"

2. For grammar questions:
   - Explain the grammar OF the target languageWelche zeile
   - All examples should be IN the target language
   Example if target is Spanish:
   Q: "What is the past tense of 'buy'?"
   A: "The Spanish past tense of 'buy' (comprar) is:
   - Compré (I bought)
   - Compraste (you bought)
   Example: Ayer compré un libro (Yesterday I bought a book)"

3. For learning requests:
   - Explain in user's language
   - All examples and practice should be IN target language
   Example if target is Spanish:
   Q: "How do I use the past tense?"
   A: "In Spanish, the simple past tense (Pretérito) is used for completed actions:
   Example: Fui al cine (I went to the cinema)"

4. When the user completes an exercise, review their answer, correct mistakes, explain why it is correct or incorrect.

IMPORTANT:
- Provide translations back to user's language in parentheses
- NEVER switch your explanation language to the target language.
- ALWAYS provide translations into the selected target language.
- Ensure that explanations are ALWAYS in the user’s input language, regardless of the selected target language
"""

# AI Language Companion 

In [5]:
system_message

' You are an intelligent language learning assisstant specialized in helping users learn new languages through interactive conversations. Your primary function is to provide accurate translations and language learning support. \nCORE TRANSLATION RULES:\n1. The dropdown language is your TARGET language - ALWAYS translate INTO this language\n2. Respond in the user\'s input language (language they write in)\n3. All examples and translations should be IN the target language\n\nHANDLING REQUESTS:\n1. For direct translations:\n   - Translate the word/phrase INTO the target language\n   - Provide target language examples\n   Example if target is Spanish:\n   Q: "What is \'buy\' in Spanish?"\n   A: "Buy = Comprar\n   Example: Voy a comprar pan (I\'m going to buy bread)"\n\n2. For grammar questions:\n   - Explain the grammar OF the target languageWelche zeile\n   - All examples should be IN the target language\n   Example if target is Spanish:\n   Q: "What is the past tense of \'buy\'?"\n   A: 

In [6]:
def language_assistant(text, target_language, history):
    messages = [
        {'role': 'system', 'content': system_message}] + history + [{'role': 'user', 
                                                                     'content': f'Please translate only the word or phrase provided into {target_language}.'
                                                                    'Additionally, provide synonyms, example sentences, or small exercises as needed. '
                                                                    'Respond in the language the user is writing in. '
                                                                    f'Text: {text}'
        f"Text: {text}"}]
    stream = openai.chat.completions.create(
    model = MODEL,
    messages = messages,
    stream = True
    )
    
    result = ''
    for chunk in stream:
        result += chunk.choices[0].delta.content or ''
        result = result.replace("```","").replace("markdown", "")

    history = history or []
    history.append({'role': 'user', 'content': text})
    history.append({'role': 'assistant', 'content': result})
    return history

## Simple UI with Gradio

In [7]:
def create_interface():
    with gr.Blocks(title = 'AI Language Companion') as interface:
        gr.Markdown("# 🌍 AI Language Companion")

        with gr.Row():
            
            with gr.Column(min_width = 600):
                inputs = gr.Textbox(label = 'Your question', placeholder = 'Type your text here..', lines = 15,                    
                                   )
                target_language = gr.Dropdown(
                    ['English', 'German', 'Spanish', 'French', 'Italian',],
                    label = 'Choose Target Language',
                    value = 'English'    
                )
                submit_button = gr.Button("Submit", variant = 'primary')

            with gr.Column():
                chatbot = gr.Chatbot(label = 'Chat History', type = 'messages',
                                              height = 600)

        with gr.Row():
            clear = gr.Button('Clear')

        submit_button.click(
            fn=language_assistant,
            inputs = [inputs, target_language, chatbot],
            outputs = chatbot,
            queue = False
        ).then(lambda: gr.update(value=''), inputs = [], outputs = inputs)
        clear.click(lambda: None, inputs = None, outputs = chatbot, queue = False)

    return interface

In [8]:
if __name__ == "__main__":
    language_companion = create_interface()
    language_companion.launch(inbrowser = True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
